# ABS Monthly Household Spending Indicator 5682

## Python set-up

In [1]:
# system imports
import sys

# analytic imports
import matplotlib.pyplot as plt
import pandas as pd

# local imports
from abs_data_capture import (
    find_rows,
    get_ABS_meta_and_data,
    get_fs_constants,
    get_meta_constants,
    get_plot_constants,
)
from plotting import clear_chart_dir, plot_growth_finalise, set_chart_dir

# pandas display settings
pd.options.display.max_rows = None
pd.options.display.max_columns = 999
pd.options.display.max_colwidth = 100

# display charts in this notebook
SHOW = False

## Get data from ABS

In [2]:
catalogue_id = "5682"
source, CHART_DIR, META_DATA = get_fs_constants(catalogue_id)
abs_dict = get_ABS_meta_and_data(catalogue_id, verbose=False)
if abs_dict is None:
    sys.exit(-1)
meta = abs_dict[META_DATA]
RECENT, plot_times, plot_tags = get_plot_constants(meta)
did_col, id_col, table_col, type_col, unit_col = get_meta_constants()

Found URL for a ZIP file on ABS web page
Retrieving zip-file from cache ...
Extracting DataFrames from the zip-file ...


## Plotting

### Set-up

In [3]:
plt.style.use("fivethirtyeight")
set_chart_dir(CHART_DIR)
clear_chart_dir(CHART_DIR)

### Headline growth

In [4]:
LONG_TITLE = 20

data_sets = {
    "01": "Monthly",
    "01Q": "Quarterly",
}
for table, period in data_sets.items():
    data = abs_dict[table]

    adjustment = "Calendar adjusted"
    prices = "Current Price"
    series_type = "Original"
    base_selector = {
        table: table_col,
        series_type: type_col,
        "household spending": did_col,
        adjustment: did_col,
        prices: did_col,
    }
    annual_selector = base_selector | {
        "Through the year percentage change": did_col,
    }
    annual_rows = find_rows(meta, annual_selector, verbose=False)
    period_selector = base_selector | {
        f"{period} percentage change": did_col,
    }
    period_rows = find_rows(meta, period_selector, verbose=False)

    for chart in [x.split(";")[1].strip() for x in annual_rows[did_col]]:
        # data wrangling
        annual_id = annual_rows[annual_rows[did_col].str.contains(chart, regex=False)][
            id_col
        ].iloc[0]
        period_id = period_rows[period_rows[did_col].str.contains(chart, regex=False)][
            id_col
        ].iloc[0]

        if annual_id not in data.columns or period_id not in data.columns:
            print(f"Bailing on {chart} where annual={annual_id} and period={period_id}")
            continue

        annual, periodic = data[annual_id], data[period_id]

        # plotting
        separator = " " if len(chart) <= LONG_TITLE else "\n"
        plot_growth_finalise(
            annual,
            periodic,
            title=f"Growth in Household Spending:{separator}{chart}",
            ylabel="Per cent",
            lfooter=f"Australia. {series_type} series. {prices}. {adjustment}. ",
            rfooter=source,
            show=SHOW,
        )

## Finished

In [5]:
# watermark
%load_ext watermark
%watermark -u -n -t -v -iv -w

Last updated: Fri May 26 2023 08:44:48

Python implementation: CPython
Python version       : 3.11.3
IPython version      : 8.13.2

pandas    : 2.0.1
matplotlib: 3.7.1
sys       : 3.11.3 | packaged by conda-forge | (main, Apr  6 2023, 09:05:00) [Clang 14.0.6 ]

Watermark: 2.3.1



In [6]:
print("Done")

Done
